In [1]:
import sklearn as skl
import matplotlib.pyplot as plt
import numpy as np
import sys
from scipy.stats import mode
import warnings
warnings.filterwarnings('ignore')

from sklearn.utils.validation import check_is_fitted
from sklearn.cluster import SpectralClustering
from scipy.io import loadmat

#Please use in all section ex

In [4]:
#part of code from https://github.com/Patrick-Louden/Ising-Model/blob/master/ising.py

import math
import random


class Ising(skl.base.BaseEstimator, skl.base.TransformerMixin):
    def __init__(self,n_spins = 200, temperature = 1.0, sparsity = 100, seed = 117, p=10):
        self.temperature = temperature
        self.n_spins = n_spins
        self.sparsity = sparsity
        self.n_C = np.ceil(np.log(n_spins)/sparsity).astype(int) #defined as in gardner, where C << logN
        self.p = p #number of patterns
        
        #choose initial condition to align with one pattern (eg. 0.8 aligned)
        self.spins = np.random.randint(2,size = (self.n_spins, 1))*2 -1 #initialize spin configuration, column vector
        self.seed = seed
    
    def getSpins(self):
        return self.spins
    
    
    def randomPatterns(self):
        pattern_tensor = np.random.randint(2,size = (1,self.n_spins,self.p))*2 -1
        return pattern_tensor
        
    def randomPatternMatrix(self):
        pattern_1d = self.randomPatterns()
        pattern_matrix = np.tile(pattern_1d, (self.n_spins,1,1))
        return pattern_matrix

    def setTemperature(self,newTemperature):
        self.temperature = newTemperature  

    def getTemperature(self):
        return self.temperature

    def setConnectivity(self):
        nums = np.zeros((self.n_spins,self.n_spins))
        nums[:self.n_C] = 1
        np.random.seed(self.seed)
        np.random.shuffle(nums)
        np.fill_diagonal(nums, 0)#no self connectivity in the model
        return nums
    
    def setWeights(self):
        pattern_matrix = self.randomPatternMatrix()
        interactions = np.multiply(pattern_matrix, np.transpose(pattern_matrix,(1,0,2)))#look at the interactions within spins
        interacions_across_patterns = np.sum(interactions, axis = 2)#sum the interactions across patterns
        C_mat = self.setConnectivity()
        J_mat = np.multiply(C_mat, interacions_across_patterns)
        return J_mat
    
    def computeField(self):
        J_mat = self.setWeights()
        field_matrix = J_mat@self.spins
        return field_matrix
    
    def getProbability(self):
        '''
        Define the probability that the spin when updated equals +1, 1-prob = spin equals -1.
        '''
       
        field = self.computeField()#get the field at time = t
        temp = self.getTemperature() #get the temperature T
        prob = (1/(np.exp(-2*field/temp)))
        return prob, temp, field
    
    def updateSpin(self):
        '''
        Update spins based on field and the probability of the spin value
        '''
        
        spin_new = np.ones((self.n_spins,1))
        prob = self.getProbability()
        rand_val = np.random.uniform(0,1) #if value is below prob, set spin to +1, else to -1
        
        for i in range(self.n_spins):
            if rand_val > prob:
                spin_new[i] = 1
        
        self.spins = np.copy(spin_new)
        
        
    def individualMean(self, idx, pattern):
        '''
        Calculate the interaction between an individual spin and the corresponding value in pattern 1
        
        Arguments:
            idx (int): index at which you want to compute the interaction
            pattern(int): pattern index with which you are looking at the overlap
           
        '''
        
        patterns = self.randomPatterns()
        individual_pattern = patterns[1,idx,pattern]
        
        spins = self.spins()
        individual_spin = spins[idx]
        
        individual_mean = np.int(individual_patter)*np.int(individual_spin)
        
        return individual_mean
    
    def averageMean(self, pattern):
        '''
        Calculate the averge mean field for all spins
        
        Arguments:
            pattern(int): pattern index with which you are looking at the overlap
        
        '''
        
        normalization = 1/self.n_spins
        avg_mean = 0
        
        for idx in range(self.n_spins):
            indiv_mean = individualMean(idx, pattern)
            avg_mean += indiv_mean
            
        avg_mean_norm = normalization*avg_mean
        
        return avg_mean_norm 




In [8]:
IS = Ising()
prob, temp, field = IS.getProbability()
spins = IS.getSpins()
J_mat = IS.setWeights()


In [10]:
print(prob)
print(temp)
print(field)

AttributeError: module 'numpy' has no attribute 'head'

In [7]:
spins

array([[-1],
       [ 1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [ 1],
       [-1],
       [ 1],
       [-1],
       [-1],
       [-1],
       [ 1],
       [ 1],
       [-1],
       [ 1],
       [ 1],
       [-1],
       [ 1],
       [ 1],
       [-1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [-1],
       [-1],
       [ 1],
       [ 1],
       [ 1],
       [-1],
       [ 1],
       [-1],
       [ 1],
       [-1],
       [-1],
       [ 1],
       [-1],
       [-1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [-1],
       [ 1],
       [ 1],
       [-1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [ 1],
       [-1],
       [-1],
       [ 1],
       [ 1],
       [-1],
       [-1],
       [ 1],
       [ 1],
       [-1],
       [ 1],
       [-1],
       [ 1],
       [-1],
       [-1],
       [ 1],
       [-1],
       [ 1],
       [ 1],

In [29]:
    #set connectivity check
    
    def setConnectivity(n, n_C):
        nums = np.zeros((n,n))
        nums[:n_C] = 1
        np.random.shuffle(nums)
        np.fill_diagonal(nums, 0)
        return nums
    
    setConnectivity(100,10)

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [74]:
prob

<bound method Ising.getProbability of Ising()>

In [82]:
#test cell
A = np.zeros((3,4,5))
A_t = np.transpose(A,(1,0,2))
print(A_t.shape)

(4, 3, 5)
